# Environment setup

In [1]:
%%capture
!pip install -q datasets==2.17.0
!pip install -q fsspec<=2023.12.0
!pip install -q evaluate

!pip install transformers
!pip install sentencepiece
!pip install accelerate
exit()

In [1]:
!wandb disabled --service

/bin/bash: line 1: wandb: command not found


In [2]:
import os

import numpy as np
import datasets
import transformers
import huggingface_hub
import matplotlib.pyplot as plt

import evaluate

from datasets import load_dataset, concatenate_datasets
from huggingface_hub import login
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    DataCollatorWithPadding,
    Trainer,
)


huggingface_hub.login(token="...")


print("Huggingface Hub Version:", huggingface_hub.__version__)
print("Transformer Version:", transformers.__version__)
print("Dataset Version:", datasets.__version__)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
Huggingface Hub Version: 0.20.3
Transformer Version: 4.38.2
Dataset Version: 2.17.0


# Processing with data

In [3]:
dataset = load_dataset("DuongTrongChi/facebook-commet-classification-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2969 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/772 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/330 [00:00<?, ? examples/s]

In [ ]:
dataset

# Looking at the Class Distribution

In [5]:
classes = ["dangerous_content", "harassment", "hate_speech", "sexaully_explixit"]
class2id = {class_:id for id, class_ in enumerate(classes)}
id2class = {id:class_ for class_, id in class2id.items()}

In [6]:
model_path = 'uitnlp/visobert'
tokenizer = AutoTokenizer.from_pretrained(model_path)

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/471k [00:00<?, ?B/s]

In [7]:
def preprocess_function(example):
   text = f"{example['text']}"
   all_labels = example['labels']
   labels = [0. for i in range(len(classes))]
   if "normal" not in all_labels:
       for label in all_labels:
           label_id = class2id[label]
           labels[label_id] = 1

   example = tokenizer(text, truncation=True, padding="max_length", max_length=256)
   example['labels'] = labels
   return example


tokenized_dataset = dataset.shuffle().map(preprocess_function)

Map:   0%|          | 0/2969 [00:00<?, ? examples/s]

Map:   0%|          | 0/772 [00:00<?, ? examples/s]

Map:   0%|          | 0/330 [00:00<?, ? examples/s]

In [8]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def sigmoid(x):
   return 1/(1 + np.exp(-x))


def compute_metrics(eval_pred):
   predictions, labels = eval_pred
   predictions = sigmoid(predictions)
   predictions = (predictions > 0.5).astype(int).reshape(-1)
   return clf_metrics.compute(predictions=predictions, references=labels.astype(int).reshape(-1))


In [9]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_path, num_labels=len(classes),
    id2label=id2class,
    label2id=class2id,
    problem_type = "multi_label_classification"
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments(
   output_dir="./facebook-commet-classification-small/",
   learning_rate=5e-5,
   per_device_train_batch_size=3,
   per_device_eval_batch_size=3,
   num_train_epochs=1,
   weight_decay=0.01,
   evaluation_strategy="epoch",
   save_strategy="epoch",
   load_best_model_at_end=True,
   report_to=["tensorboard"]
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

trainer.train()


pytorch_model.bin:   0%|          | 0.00/390M [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at uitnlp/visobert and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.226100,0.160514,0.942681,0.755187,0.829787,0.692893


TrainOutput(global_step=990, training_loss=0.1929703991822522, metrics={'train_runtime': 190.0496, 'train_samples_per_second': 15.622, 'train_steps_per_second': 5.209, 'total_flos': 390595375552512.0, 'train_loss': 0.1929703991822522, 'epoch': 1.0})

# Push to huggingface hub

In [10]:
repo_id = "DuongTrongChi/facebook-commet-classification-small"
tokenizer.push_to_hub(repo_id)
trainer.push_to_hub(repo_id)

README.md:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/390M [00:00<?, ?B/s]

events.out.tfevents.1712812908.4b0604d42325.1400.0:   0%|          | 0.00/6.00k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DuongTrongChi/facebook-commet-classification-small/commit/f2fcf3b35e15b4ea060290e266114280c11779de', commit_message='DuongTrongChi/facebook-commet-classification-small', commit_description='', oid='f2fcf3b35e15b4ea060290e266114280c11779de', pr_url=None, pr_revision=None, pr_num=None)

# Inferrences

In [ ]:
from transformers import pipeline

pipe = pipeline("text-classification", model="DuongTrongChi/facebook-commet-classification-filter-v4", return_all_scores=True)

2024-04-10 06:59:32.775741: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 06:59:32.775912: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 06:59:32.981985: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/990 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/390M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/471k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
def predict(text):
    res = pipe(text)[0]
    results = dict()
    for x in res:
        results[x['label']] = x['score']
    return results


print(predict("xin chào em"))

{'dangerous_content': 0.0006770377513021231, 'harassment': 0.0008820039802230895, 'hate_speech': 0.0010275174863636494, 'sexaully_explixit': 0.0019994559697806835}
